In [1]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen as url
import urllib
import selenium
import logging

In [2]:
nba_url = "https://www.basketball-reference.com/"
uc = url(nba_url)
nba_pg = uc.read()

In [3]:
logging.basicConfig(filename = "nba.log" ,level = logging.INFO)
logging.info("Logging information")
logging.warning("Logging warnings ")
logging.error("Logging errors")
logging.basicConfig(filename = "Scrap.log" , level = logging.DEBUG , format ='%(asctime)s %(levelname)s %(message)s' )

In [4]:
#Gets data from the webpage
uc.close()
nba_html = bs(nba_pg,"html.parser")
bb = nba_html.find("div",{"id":"nav"})
ply_link = "https://www.basketball-reference.com/"+bb.ul.li.a['href']
ply_list = requests.get(ply_link)
ply_list.encoding= 'utf-8'
ply_html = bs(ply_list.text,'html.parser')
bb1 = ply_html.find("div",{"id":"content"})
ply_bb = bb1.find('ul')

In [5]:
play_names=ply_bb.text

In [6]:
#Extarcts names of individual players from the string generated
p_l =[]
p = []
s =''
i=0
while i in range(len(play_names)):
    
    if play_names[i]!='\xa0':
        s = s+play_names[i]
        i = i+1
    else:
        p_l.append(s)
        i = i+1
        
        s = ''

for i in p_l:
    l =len(i)
    ind = p_l.index(i)
    r = i[:l-1]
    i = r   
    p_l[ind] = r
         
play_names = p_l

In [7]:
#Extracts links of the pages where the details of players are recorded
ply_bb = bb1.find_all("a",text=True)
ply_link_bb = ply_bb[1:]
links=[]
for i in range(len(ply_link_bb)):
    p_l=ply_link_bb[i]['href']
    link="https://www.basketball-reference.com/"+ply_link_bb[i]['href']
    links.append(link)

In [8]:
#Players stats are extracted
ind_value=0
player_details_nba=[]
for link in links:
    try:
        evry_ply_det_link = requests.get(link)
        evry_ply_det_link.encoding= 'utf-8'
        evry_ply_det = bs(evry_ply_det_link.text,'html.parser')
        ply_bb1 = evry_ply_det.find("div",{"class":"stats_pullout"})
        link_count = link_count+1
        player_details =ply_bb1.find_all("p",text =True)
        if '*' in play_names[ind_value]:
            details =[]
            for i in range(0,len(player_details)-1):
                details.append(player_details[i].text) 
                p_d = details[1:]
                det_ppl=[]
                for i in p_d:
                    if i == '-':
                        n = 0
                        det_ppl.append(n)
                    else:
                        n = float(i)
                        det_ppl.append(n)
            player_details_nba.append(det_ppl)
        else:    
            details =[]
            for i in range(0,len(player_details)-1):
                details.append(player_details[i].text) 
                p_d = details[1::2]
                plyer_det=p_d[1:]
                det_ppl=[]
                for i in plyer_det:
                    if i =='-':
                        n=0 
                        det_ppl.append(n)
                    else:
                        n = float(i)
                        det_ppl.append(n)
            player_details_nba.append(det_ppl)
        ind_value=ind_value+1
        
    
    except Exception as e:
        logging.info(e)

In [9]:
#Extracts win share data from the website
ind_value=0

i=0
ws_det=[]
for link in links:
    try:
            
        evry_ply_det_link = requests.get(link)
        evry_ply_det_link.encoding= 'utf-8'
        evry_ply_det = bs(evry_ply_det_link.text,'html.parser')
        ply_bb1 = evry_ply_det.find("div",{"class":"stats_pullout"})
        ply_ws = ply_bb1.find("div",{"class":"p3"})
        ply_ws_det = ply_ws.find_all("p",text=True)
        ws = ply_ws_det[1].text
        ws_det.append(float(ws))
        
    except Exception as e:
        logging.info(e)

In [10]:
#Adds players name to the stats 
ind = 0
for i in player_details_nba:
    n = play_names[ind]
    i.insert(0,str(n))
    ind=ind+1

In [11]:
import pandas as pd

In [12]:
#Creates the dataframe called df which stores the players data in structured 
df = pd.DataFrame(player_details_nba, columns=['Player name','Number of games played','Points scored','Total rebounds','Assists','Field goal percentage','3-point field goal percentage','Free throw percentage','Effective field goal percentage','Player efficiency rate'])
df['Winning shares'] = ws_det

In [13]:
df

,Player name,Number of games played,Points scored,Total rebounds,Assists,Field goal percentage,3-point field goal percentage,Free throw percentage,Effective field goal percentage,Player efficiency rate,Winning shares
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,273.4
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.7
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.5
...,...,...,...,...,...,...,...,...,...,...,...
145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.2
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.2
147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.1
148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2
